In [1]:
import torch
import numpy as np

In [3]:
from utils.make_data import autoregressive, generate_autoregressive_forecast_dataset
from models.conformal import nonconformity, cover, ConformalForecaster

In [4]:
train_dataset = generate_autoregressive_forecast_dataset(n_samples=1000, seq_len=100, horizon=10)
calibration_dataset = generate_autoregressive_forecast_dataset(n_samples=1000, seq_len=100, horizon=10)
test_dataset = generate_autoregressive_forecast_dataset(n_samples=100, seq_len=100, horizon=10)

In [5]:
model = ConformalForecaster(embedding_size=8, horizon=10, error_rate=0.05)

In [6]:
model.fit(train_dataset, calibration_dataset, epochs=100, lr=0.01, batch_size=100)

Epoch: 0	Train loss: 117.00947952270508
Epoch: 50	Train loss: 15.443363380432128


In [10]:
test_dataset = generate_autoregressive_forecast_dataset(n_samples=100, seq_len=100, horizon=10)
model.eval()
c = []
for sequences, target in test_dataset:
    sequences = sequences.unsqueeze(dim=0)
    out = model(sequences).squeeze()
    pred = torch.vstack([out - model.critical_calibration_scores,
                         out + model.critical_calibration_scores]).T
    c.append(cover(pred, target))
print('Achieved coverage: {}'.format(np.mean(c)))

Achieved coverage: 0.71
